In [1]:
import numpy as np

from lcc.dataset import create_lcc_dataset_k1_t1_scalar

from probabilistic_classifier.dataset import create_multiclass_conditional_dataset
from probabilistic_classifier.estimate import estimate_mi_for_multiclass_classification
from probabilistic_classifier.train import train_multiclass_classifier

In [ ]:
# create the basis dataset
prime = 5
data_range = 2
num_of_samples = 800000
weight = 1
dataset = create_lcc_dataset_k1_t1_scalar(prime, data_range, num_of_samples, weight)

In [ ]:
dataset = np.concatenate([dataset[:, :3], dataset[:, 4:]], axis=1)

In [ ]:
# create the joint and marginal datasets
x_idx, y_idx, z_idx = [0, 1], [3, 4, 5], [2]
joint_data, joint_label, all_marginal_data, all_marginal_label, marginal_y_joint_xz_data, marginal_y_joint_xz_label, marginal_x_joint_yz_data, marginal_x_joint_yz_label = create_multiclass_conditional_dataset(dataset, x_idx, y_idx, z_idx)
data, label = np.concatenate([joint_data, all_marginal_data, marginal_y_joint_xz_data, marginal_x_joint_yz_data]), np.concatenate([joint_label, all_marginal_label, marginal_y_joint_xz_label, marginal_x_joint_yz_label])
randomize_idx = np.random.permutation(np.arange(4 * num_of_samples))
data, label = data[randomize_idx], label[randomize_idx]

In [ ]:
# define model parameters
num_input_features = len(x_idx) + len(y_idx) + len(z_idx)
hidden_size_arr = [256, 256, 256]
lr = 0.001

In [ ]:
num_of_outer_iteration = 20
num_of_inner_iteration = 50
batch_size = 4096

# iterate over many times
outer_running_loss = []
outer_running_loss_avg = []
ldr_estimations = []

for outer_iter in range(num_of_outer_iteration):
    print('################################################################')
    (model, inner_running_loss, inner_running_loss_avg, num_of_joint, num_of_all_marginal, 
     num_of_marginal_y_joint_xz, num_of_marginal_x_joint_yz) = train_multiclass_classifier(data, label, num_input_features, hidden_size_arr, lr, num_of_inner_iteration, batch_size, outer_iter, print_progress=True, save_avg=500)
    outer_running_loss.append(inner_running_loss)
    outer_running_loss_avg.append(inner_running_loss_avg)
    
    ## estimate cmi
    curr_ldr = estimate_mi_for_multiclass_classification(model, joint_data, num_of_joint, num_of_all_marginal, num_of_marginal_y_joint_xz, num_of_marginal_x_joint_yz)
    print('trial: {}, ldr: {}'.format(outer_iter + 1, curr_ldr.item()))
    print('################################################################\n')
    ldr_estimations.append(curr_ldr.item())
    
print('final estimations:\n\tldr: {}'.format(np.mean(ldr_estimations)))